In [1]:
import argparse
import logging
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import model_interface as mi
import tflite_utils

In [2]:
DEFAULT_INPUTS = 32
DEFAULT_OUTPUTS = 16
DEFAULT_K_H = 3
DEFAULT_K_W = DEFAULT_K_H
DEFAULT_HEIGHT = 5
DEFAULT_WIDTH = DEFAULT_HEIGHT


In [3]:
def generate_data(height, width, inputs):
    quant_data = np.float32(
        np.random.uniform(0, 1, size=(10, height, width, inputs)))
    x_test_float = np.concatenate(
        [np.zeros((1, height, width, inputs), dtype=np.float32),
         quant_data[:3, :, :, :]],  # pylint: disable=unsubscriptable-object
        axis=0)
    return x_test_float, quant_data


In [4]:
class Conv2dDeepinDeepoutRelu(mi.KerasModel):
    def build(self, K_h, K_w, height, width):
        input_dim = self.input_dim
        output_dim = self.output_dim
        # Env
        tf.keras.backend.clear_session()
        tflite_utils.set_all_seeds()
        # Building
        model = tf.keras.Sequential(name=self.name)
        model.add(layers.Conv2D(
            filters=output_dim,
            kernel_size=(K_h, K_w),
            padding='same',
            input_shape=(height, width, input_dim)))
        # Compilation
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        # Add to dict
        self.models[self.name] = model
        # Show summary
        model.summary()
        return model

    def train():
        pass

    def prep_data(self, height, width):
        self.data['export_data'], self.data['quant'] = generate_data(
            height, width, self.input_dim)

    def gen_test_data(self, height, width):
        if not self.data:
            self.prep_data(height, width)


In [5]:
def main(input_dim=DEFAULT_INPUTS,
         output_dim=DEFAULT_OUTPUTS,
         K_h=DEFAULT_K_H,
         K_w=DEFAULT_K_W,
         height=DEFAULT_HEIGHT,
         width=DEFAULT_WIDTH):
    assert input_dim % 32 == 0, "# of input channels must be multiple of 32"
    assert output_dim % 16 == 0, "# of output channels must be multiple of 16"
    # Instantiate model
    test_model = Conv2dDeepinDeepoutRelu(
        'conv2d_deepin_deepout_relu', Path('.'), input_dim, output_dim)
    # Build model and compile
    test_model.build(K_h, K_w, height, width)
    # Generate test data
    test_model.gen_test_data(height, width)
    # Populate converters
    # test_model.populate_converters() #breaks in xcore
    test_model.to_tf_float()
    test_model.to_tf_quant()
    test_model.to_tf_stripped()
    # Convert and save, will break in stripped
    test_model.convert_and_save()

In [17]:

!rm -rf test_data
!rm -rf models

In [6]:

input_dim = DEFAULT_INPUTS
output_dim = DEFAULT_OUTPUTS
test_model = Conv2dDeepinDeepoutRelu(
        'conv2d_deepin_deepout_relu', Path('.'), input_dim, output_dim)
print(test_model.models)

{'data_dir': PosixPath('test_data'), 'models_dir': PosixPath('models')}


In [7]:
width = DEFAULT_WIDTH
height = DEFAULT_HEIGHT
K_h = DEFAULT_K_H
K_w = DEFAULT_K_W
test_model.build(K_h, K_w, height, width)

Model: "conv2d_deepin_deepout_relu"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 5, 5, 16)          4624      
Total params: 4,624
Trainable params: 4,624
Non-trainable params: 0
_________________________________________________________________


In [8]:
test_model.gen_test_data(height, width)

In [9]:
test_model.data.keys()

dict_keys(['export_data', 'quant'])

In [10]:
test_model.to_tf_float()

In [11]:
test_model.to_tf_quant()

In [12]:
test_model.to_tf_stripped()

In [13]:
test_model.to_tf_xcore() # Expected break

TypeError: can't pickle _thread.RLock objects

In [14]:
# Checks
print(test_model.data.keys())
print(test_model.models.keys())

dict_keys(['export_data', 'quant'])
dict_keys(['data_dir', 'models_dir', 'conv2d_deepin_deepout_relu', 'model_float', 'model_quant', 'model_stripped'])


In [15]:
test_model.populate_converters()

TypeError: can't pickle _thread.RLock objects

In [16]:
test_model.convert_and_save()

KeyError: 'model_xcore'